In [1]:
import os
import glob
import matplotlib
import numpy
import numpy as np
import cv2 
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import random
from tensorflow import convert_to_tensor

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
BASE_PATH = os.getcwd()
IMAGES_PATH = BASE_PATH + os.sep + "Images" + os.sep
IMAGES_FOLDERS = [(lambda path: IMAGES_PATH + path)(path) for path in os.listdir(IMAGES_PATH)]
IMAGES_FOLDERS = IMAGES_FOLDERS[1:]
IMAGERS_DATA = [(lambda path: IMAGES_FOLDERS[0] + path)(path) for path in os.listdir(IMAGES_FOLDERS[0])]

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: 'D:\\PycharmProjects\\PandasLearn\\Images\\'

In [ ]:
data = []
labels = []

for num in range(0, 43):
    path = os.path.join(IMAGES_PATH, str(num))
    imagePaths = os.listdir(path)
    for img in imagePaths:
        try:
            if(img[0] == "G" or img[0] == " " or img == ""):
                print(img)
                continue
            image = Image.open(path + os.sep + img)
            image = image.resize((30,30))
            image = img_to_array(image)
            data.append(image)
            labels.append(num)
        except:
            continue

In [ ]:
print(data)
data = np.array(data)
labels = np.array(labels)
print(data.shape, labels.shape)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
def cnt_img_in_classes(labels):
    count = {}
    for i in labels:
        if i in count:
            count[i] += 1
        else:
            count[i] = 1
    return count


samples_distribution = cnt_img_in_classes(y_train)

In [ ]:
def diagram(count_classes):
    plt.bar(range(len(count_classes)), sorted(list(count_classes.values())), align='center')
    plt.xticks(range(len(count_classes)), sorted(list(count_classes.keys())), rotation=90, fontsize=7)
    plt.show()


diagram(samples_distribution)

In [ ]:
def aug_images(images, p):
    from imgaug import augmenters as iaa
    augs = iaa.SomeOf((2, 4),
                      [
                          iaa.Crop(px=(0, 4)),
                          iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}),
                          iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),
                          iaa.Affine(rotate=(-45, 45)),
                          iaa.Affine(shear=(-10, 10))
                      ])

    seq = iaa.Sequential([iaa.Sometimes(p, augs)])
    res = seq.augment_images(images)
    return res

def augmentation(images, labels):
    min_imgs = 500
    classes = cnt_img_in_classes(labels)
    for i in range(len(classes)):
        if classes[i] < min_imgs:
            add_num = min_imgs - classes[i]
            imgs_for_augm = []
            lbls_for_augm = []
            for j in range(add_num):
                im_index = random.choice(np.where(labels == i)[0])
                imgs_for_augm.append(images[im_index])
                lbls_for_augm.append(labels[im_index])
            augmented_class = aug_images(imgs_for_augm, 1)
            augmented_class_np = np.array(augmented_class)
            augmented_lbls_np = np.array(lbls_for_augm)
            imgs = np.concatenate((images, augmented_class_np), axis=0)
            lbls = np.concatenate((labels, augmented_lbls_np), axis=0)
    return (images, labels)


X_train, y_train = augmentation(X_train, y_train)

augmented_samples_distribution = cnt_img_in_classes(y_train)
diagram(augmented_samples_distribution)

In [ ]:
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

print(y_train)
print(y_test)

In [ ]:
class Net:
  @staticmethod
  def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    if K.image_data_format() == 'channels_first':
      inputShape = (depth, height, width)
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=inputShape))
    model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(classes, activation='softmax'))
    return model

In [ ]:
epochs = 25
model = Net.build(width=30, height=30, depth=3, classes=43)
print(np.shape(X_train))
print(np.shape(y_train))
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=64, epochs=epochs)

In [ ]:
plt.figure()
N = epochs
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()


In [ ]:
images = []
img = Image.open(BASE_PATH + os.sep + "test38.ppm")
img.show()
image = img.resize((30,30), 3)
images.append(img_to_array(image))
print(np.shape(images))
X_test=np.array(images)
pred = model.predict_classes(X_test)
road_signs = ["Ограничение скорости 20 км/ч", "Ограничение скорости 30 км/ч", "Ограничение скорости 50 км/ч", "Ограничение скорости 60 км/ч", "Ограничение скорости 70 км/ч", "Ограничение скорости 80 км/ч", "Конец зоны ограничения 80 км/ч", "Ограничение скорости 100 км/ч", "Ограничение скорости 120 км/ч", "Обгон запрещен", "Обгон грузовым автомобилям запрещен", " ", "Главная дорога", "Уступи дорогу", "Движение без остановки запрещено", "Движение запрещено", "Движение грузовых автомобилей запрещено", "Въезд запрещен", "Прочие опасности", "Опасный поворот(левый)", "Опасный поворот(правый)", "Опасные повороты (с первым поворотом налево)", "Неровная дорога", "Скользкая дорога", "Сужение дороги", "Дорожные работы", "Светофорное регулирование", "Пешеходный переход", "Дети", "Пересечение с велосипедной дорожкой или велопешеходный переход", "Субботние, воскресные и праздничные дни", "Дикие животные", "Конец зоны всех ограничений", "Движение направо", "Движение налево", "Движение прямо", "Движение прямо или направо", "Движение прямо или налево", "Объезд препятствия справа", "Объезд препятствия слева", "Круговое движение", "Конец зоны запрещения обгона", "Конец зоны запрещения обгона грузовым автомобилям"]
print(road_signs[int(pred)])

In [ ]:
directory = "your path "
for infile in glob.glob("*.jpg"):
    file, ext = os.path.splitext(infile)
    im = Image.open(infile)
    im.save(file + ".ppm", "PPM")

In [ ]:
cap = cv2.VideoCapture(0)

while(True): 
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Video', frame)
    cv2.imshow('frame', gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()